# BioExplorer - Neurons soma report
![](../bioexplorer_neurons_banner.png)

### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer

url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()
status = be.reset_scene()

### Load neurons

In [ ]:
assembly_name = 'Neurons'
load_morphologies = False
radius_multiplier = 1.0
sql_node_filter = 'guid%10=0'
if not load_morphologies:
    sql_node_filter=''
    radius_multiplier = 5.0

be.remove_assembly(assembly_name)
neurons_assembly = be.add_assembly(assembly_name)
neurons_model = be.add_neurons(
    assembly_name=assembly_name,
    population_name='o1',
    morphology_representation=be.MORPHOLOGY_REPRESENTATION_SECTION,
    load_axon=False, load_basal_dendrites=load_morphologies, load_apical_dendrites=load_morphologies,
    simulation_report_id=0,
    radius_multiplier=radius_multiplier,
    sql_node_filter=sql_node_filter
)

In [ ]:
import seaborn as sns
def set_morphology_materials(model_id, palette_name):
    colors = list()
    shading_modes = list()
    cast_user_datas = list()
    
    material_ids = be.get_material_ids(model_id)['ids']
    palette = sns.color_palette(palette_name, len(material_ids))

    for i in range(len(material_ids)):
        colors.append(palette[i])
        shading_modes.append(be.SHADING_MODE_NONE)
        cast_user_datas.append(True)
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=colors, specular_colors=colors,
        shading_modes=shading_modes, cast_user_datas=cast_user_datas
    )

In [ ]:
model_ids = be.get_model_ids()['ids']
model_id = model_ids[len(model_ids)-1]
set_morphology_materials(model_id, 'Set3')

In [ ]:
status = be.reset_camera()

In [ ]:
status = core.set_renderer(
    background_color=[0,0,0],
    current='advanced', 
    samples_per_pixel=1,
    subsampling=4,
    max_accum_frames=64)
params = core.AdvancedRendererParams()
params.gi_ray_length = 1000.0
params.shadow_intensity = 1.0
params.soft_shadow_strength = 1.0
params.epsilon_multiplier = 1.0
params.max_ray_depth = 1
params.show_background = False
params.main_exposure = 1.0
status = core.set_renderer_params(params)
status = core.set_renderer()

In [ ]:
status = core.set_renderer(
    background_color=[0,0,0],
    current='bio_explorer_voxel', 
    samples_per_pixel=1,
    subsampling=4,
    max_accum_frames=64)
params = core.BioExplorerVoxelRendererParams()
params.alpha_correction = 0.1
params.simulation_threshold = 0.3
params.main_exposure = 1.0
status = core.set_renderer_params(params)
status = core.set_renderer()

In [ ]:
core.set_animation_parameters(current=0)

In [ ]:
from bioexplorer import MovieMaker
from tqdm import tqdm

mm =MovieMaker(be)
frame = 0
for i in tqdm(range(0, 35000, 5)):
    core.set_animation_parameters(current=i)
    mm.create_snapshot(
        renderer='bio_explorer_voxel', base_name='%05d' % frame, path='/tmp',
        samples_per_pixel=4, size=[2160, 3840], show_progress=False,
    )
    frame += 1

In [ ]:
from bioexplorer import TransferFunction
model_ids = be.get_model_ids()['ids']
model_id = model_ids[len(model_ids)-1]
tf = TransferFunction(
    bioexplorer=be, model_id=model_id,
    name='rainbow', size=9, alpha=1.0,
    value_range=[-80, -10])